In [ ]:
import os
import json
import math
import datetime

from pathlib import Path

from shapely import wkt, Point
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

from s2geometry import *
import s2sphere

import trackintel as ti
from trackintel.preprocessing.triplegs import generate_trips
from trackintel.analysis.tracking_quality import temporal_tracking_quality, _split_overlaps

from IPython.utils.text import columnize
import folium

In [ ]:
print('** S2 Functions **\n')
print(columnize(dir(s2), displaywidth=100))

# Bin locations

In [ ]:
locs = pd.read_csv(os.path.join(".", "data", f"locations.csv"))
locs["center"] = locs["center"].apply(wkt.loads)
locs = gpd.GeoDataFrame(locs, geometry="center", crs="EPSG:4326")

In [ ]:
# read and simplify swiss boundary 
swissBoundary = gpd.read_file(os.path.join(".", "data", "swiss", "swiss_1903+.shp")).to_crs("EPSG:4326")

# get the geometry
swiss_polygon = swissBoundary.geometry.iloc[0]

print(len(swiss_polygon.exterior.coords))

# simplify
swiss_polygon_simplify = swiss_polygon.simplify(0.01, preserve_topology=False)

print(len(swiss_polygon_simplify.exterior.coords))

In [ ]:
coords = swiss_polygon_simplify.exterior.coords
coords[0]

In [ ]:
# construct the S2Polygon object

point_ls = []

for xx, yy, _ in coords:
    point_ls.append(S2LatLng.FromDegrees(yy, xx).ToPoint())

loop = S2Loop(point_ls)
if ~loop.IsNormalized():
    loop.Normalize()
swiss_region = S2Polygon(loop)

In [ ]:
# check the validity, should be smaller than 2*pi
swiss_region.GetArea()

In [ ]:
# get the covering

coverer = s2.S2RegionCoverer()
# atm we select 13
coverer.set_min_level(13)
coverer.set_max_level(13)

covering = coverer.GetCovering(swiss_region)

In [ ]:
len(covering)

In [ ]:
# get the location gdf and covering cell

cell_Ids = []
row_ls = []
for i in range(len(covering)):
    cell_Ids.append(s2sphere.CellId(id_=covering[i].id()))

    row = {}
    row["loc_id"] = covering[i].id()
    row["geometry"] = Point(covering[i].ToLatLng().lng().degrees(), 
                          covering[i].ToLatLng().lat().degrees())
    row_ls.append(row)

all_location_gdf = gpd.GeoDataFrame(row_ls, geometry = "geometry", crs="EPSG:4326")

In [ ]:
all_location_gdf

In [ ]:
# project locations into s2 cells
def get_loc_id(row):
    cell = s2sphere.Cell.from_lat_lng(s2sphere.LatLng.from_degrees(row.y, row.x))
    return cell.id().parent(13).id()

locs["s2_id"] = locs["center"].apply(get_loc_id)

In [ ]:
locs

### Save results

In [ ]:
# all location covering switzerland
all_location_gdf.to_csv(os.path.join(".", "data", f"all_locations.csv"))
# original location with s2 cell ids
locs.to_csv(os.path.join(".", "data", f"locs_s2.csv"))

## Plotting for correctness check

In [ ]:
def get_bbox(coords):
    lat_min = 1e7
    lat_max = -1e7
    lon_min = 1e7
    lon_max = -1e7
    for lat, lon in coords:
        lat_min = min(lat_min, lat)
        lat_max = max(lat_max, lat)
        lon_min = min(lon_min, lon)
        lon_max = max(lon_max, lon)
    return [(lat_min, lon_min), (lat_max, lon_max)]

def s2_to_geo_boundary(cell_id):
    cell = s2sphere.Cell(cell_id)
    boundary = []
    for k in range(4):
        ll = s2sphere.LatLng.from_point(cell.get_vertex(k))
        boundary.append((ll.lat().degrees, ll.lng().degrees))
    return boundary

def plot_s2(cell_id):
    print(cell_id)
    return plot_geometries([s2_to_geo_boundary(cell_id)])

def plot_s2cells(cell_ids):
    polygons = []
    for cell_id in cell_ids:
        # print(cell_id)
        polygons.append(s2_to_geo_boundary(cell_id))
    return plot_geometries(polygons)

def plot_geometries(polygons=[], points=[]):
    f = folium.Figure(width=600, height=300)
    m = folium.Map(
        tiles='https://tiles.stadiamaps.com/tiles/osm_bright/{z}/{x}/{y}{r}.png',
        attr='(C) Stadia Maps, (C) OpenMapTiles (C) OpenStreetMap contributors',
        zoom_start=20, max_zoom=20).add_to(f)
    for polygon in polygons:
        folium.Polygon(polygon, color='#ff0000', opacity=1).add_to(m)
    for point in points:
        folium.Marker(point, color='#0000ff', opacity=1).add_to(m)
    bb = get_bbox([c for p in polygons for c in p])
    m.fit_bounds(bb)
    return m

In [ ]:
plot_s2cells(cell_Ids)

In [ ]:
loc_cell_ls = []

for center in locs["center"].values:
    cell = s2sphere.Cell.from_lat_lng(s2sphere.LatLng.from_degrees(center.y, center.x))
    loc_cell_ls.append(cell.id().parent(11))

In [ ]:
plot_s2cells(loc_cell_ls)

In [ ]:
# comparing all locations and existing locations
len(covering), len(locs["s2_id"].unique())